In [ ]:
import json

all_tools = set()
with open("/home/snrobot/lin/ComplexFuncBench/data/ComplexFuncBench.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        id = data["id"]
        category = id.split("_")[0]
        funcs = data["functions"]
        for tool in funcs:
            all_tools.add(json.dumps(tool))

output_data = {'function': [json.loads(tool) for tool in all_tools]}
with open("/home/snrobot/lin/GraphLinkTools/Tools/All_Tools.json", 'w', encoding='utf-8') as outfile:
    json.dump(output_data, outfile, ensure_ascii=False, indent=4)
print(len(all_tools))

In [ ]:
import json

unique_tools = set()

with open("/home/snrobot/lin/GraphLinkTools/Tools/All_Tools.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
    tools = data['function']
    for tool in tools:
        unique_tools.add(tool["name"])
print(unique_tools)
print(len(unique_tools))

        
unique_tools_rapidapi = set()
with open("/home/snrobot/lin/ComplexFuncBench/utils/tool_info.json", 'r', encoding='utf-8') as f:
    tool_info = json.load(f)
    tools = tool_info["booking-com15"]['name_to_url']
    unique_tools_rapidapi = set(tools.keys())
print(len(unique_tools_rapidapi))
print(unique_tools_rapidapi - unique_tools)
    



{'Search_Attraction_Location', 'Get_Hotel_Reviews(Tips)', 'Search_Hotels_By_Coordinates', 'Get_Hotel_Reviews(Tips)_Sort_Option', 'Search_Hotel_Destination', 'Get_Seat_Map', 'Taxi_Search_Location', 'Get_Flight_Details', 'Get_Min_Price', 'Vehicle_Supplier_Review', 'Payment_features_of_the_Hotel', 'Property_Children_Policies', 'Get_Hotel_Review_Scores', 'Get_Question_And_Answer', 'Get_Attraction_Details', 'Get_Hotel_Details', 'Vehicle_Details', 'Get_Packages', 'Booking_Summary', 'Search_Flight_Location', 'Get_Description_And_Info', 'Get_Availability', 'Location_to_Lat_Long', 'Get_Room_Availability', 'Search_Attractions', 'Search_Flights_Multi_Stops', 'Search_Car_Rentals', 'Get_Room_List_With_Availability', 'Search_Flights', 'Vehicle_Supplier_Details', 'Vehicle_Supplier_Ratings', 'Get_Filter', 'Search_Car_Location', 'Search_Taxi', 'Get_Availability_Calendar', 'Get_Min_Price_Multi_Stops', 'Get_Hotel_Policies', 'Search_Hotels', 'Get_Popular_Attraction_Near_By', 'Get_Nearby_Cities'}
40
48
{'G

In [21]:
import json

Apis = {}
with open("/home/snrobot/lin/GraphLinkTools/Tools/Apis.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
    Apis = data['functions']
    
with open("/home/snrobot/lin/GraphLinkTools/Tools/All_Tools.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
    Tools = data['function']
    
tool_descriptions = {tool["name"]: tool["description"].split("EndPoint: ")[0] for tool in Tools}
tool_endpoint = {tool["name"]: tool["description"].split("EndPoint: ")[1] for tool in Tools}

for api in Apis:
    tool_name = api["name"]
    if tool_name in tool_descriptions:
        api["func_description"] = tool_descriptions[tool_name]
        api["endpoint"] = tool_endpoint[tool_name]
    
funcs = {"functions":Apis}
with open("/home/snrobot/lin/GraphLinkTools/Tools/Apis.json", 'w', encoding='utf-8') as f:
    json.dump(funcs, f, ensure_ascii=False, indent=4)
    
    

In [22]:
with open("/home/snrobot/lin/ComplexFuncBench/utils/tool_info.json", 'r', encoding='utf-8') as f:
    tool_info = json.load(f)
    tools = tool_info["booking-com15"]['name_to_url']

for key in tools:
    url = tools[key]
    last_part = url.split('https://booking-com15.p.rapidapi.com')[-1]  # 按 '/' 分割并取最后一段
    tools[key] = last_part

Apis = {}
with open("/home/snrobot/lin/GraphLinkTools/Tools/Apis.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
    Apis = data['functions']
    
for api in Apis:
    tool_name = api["name"]
    if tool_name in tools:
        api["endpoint"] = tools[tool_name]
        
funcs = {"functions":Apis}
with open("/home/snrobot/lin/GraphLinkTools/Tools/Apis(1).json", 'w', encoding='utf-8') as f:
    json.dump(funcs, f, ensure_ascii=False, indent=4)





In [2]:
# 从测试集中获取function_call和observation
import json

input_file = "/home/snrobot/lin/ComplexFuncBench/data/ComplexFuncBench.jsonl"
output_file = "/home/snrobot/lin/GraphLinkTools/Tools/Static_Response(1).json"


with open(input_file, "r") as f:
    datas = [json.loads(line) for line in f]
    
results = []
for data in datas:
    conversations = data["conversations"]
    query = conversations[0]["content"]
    i = 0
    while i < len(conversations):
        if conversations[i]["role"] == "assistant" and "function_call" in conversations[i]:
            calls = conversations[i]["function_call"]
            j = i + 1
            while j < len(conversations):
                if conversations[j]["role"] == "observation":
                        observations = conversations[j]["content"]
                        break
                j += 1
            else:
                observations = None
            
            for call, obs in zip(calls, observations):
                results.append({
                    "related_query": query,
                    "function_name": call["name"],
                    "arguments": call["arguments"],
                    "observation": obs
                }) 
        i += 1
with open(output_file, 'w', encoding='utf-8') as out_f:
    json.dump(results, out_f, ensure_ascii=False, indent=4)

print(f"共提取 {len(results)} 条 function_call 与 observation 映射，已保存至 {output_file}")
    


共提取 5070 条 function_call 与 observation 映射，已保存至 /home/snrobot/lin/GraphLinkTools/Tools/Static_Response(1).json


In [ ]:
# 持久化存储文件
import json
import glob
import os

# 持久化存储文件
mapping_file = "/home/snrobot/lin/GraphLinkTools/Tools/mapping/api_mapping.json"

def load_mapping():
    try:
        with open(mapping_file, "r") as f:
            return json.load(f)
    except FileNotFoundError:
        return {}
    except json.JSONDecodeError:
        print(f"Error: {mapping_file} is not a valid JSON file.")
        return {}
    except Exception as e:
        print(f"Error loading mapping file: {e}")
        return {}

def save_mapping(mapping):
    try:
        with open(mapping_file, "w") as f:
            json.dump(mapping, f, indent=4)
    except Exception as e:
        print(f"Error saving mapping file: {e}")

def get_or_assign_id(name):
    mapping = load_mapping()
    if name not in mapping:
        # 生成新的唯一 ID
        existing_ids = set(mapping.values())
        new_id = max(existing_ids, default=-1) + 1  # 从最大 ID + 1 开始
        mapping[name] = new_id
        save_mapping(mapping)       
    return mapping[name]

# 示例：为每个文件获取或分配唯一 ID
api_folder_path = "/home/snrobot/lin/GraphLinkTools/Tools/func2"
info_files = glob.glob(os.path.join(api_folder_path, '*.json'))
api_ids = {}

for file_path in info_files:
    print(file_path)
    break
    try:
        notebook = json.load(open(file_path, 'r', encoding='utf-8'))
        name = notebook['shorten']['api']['name']
        api_id = get_or_assign_id(name)
        api_ids[name] = api_id
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

print(api_ids)

/home/snrobot/lin/GraphLinkTools/Tools/func2/Search_Car_Rentals.json
{}


In [6]:
len("d7699_H4sIAAAAAAAA_1WQ3U7jMBCFnwbu4nTcpE6QrFW2LVA2P0CSQntjpa6bhnbxKjYk7dNjcNUFjTXnfEeakTVbrf-pK9fd7Jt6q5XzplAttawrLRCXf91Na9pKyl3zWrtV07pRESfTeBCm2RywaS64jil-tfgleu2oltPLZiVQRZ0wDK1VnAYn19IR8qbhcpnlNuJSUw_54JPstnweRzZs6c3d7e8bO7-mybjr4kmkkrzr48mUnNkqSSaqz46LQXYs-7RY3-fXJX7c1SQp0nly6Pr05To1czgpO5IVs8-5_mu14JwCQh4J4YtlpejQOq4pHli71jSP79Ll3KKmQLxwZP_fUxgOA_9Sib3gupGvf8SBlhEOiGOOJN-x5xSsjADwzyAMfuAIHHPOM2JszPcACAT_gws8hsPel4obdzGMzKvZAJ3L8PbE8FmGK1ZE6BkjgBTFGZotUJmbeMWCByOcBYWRNYOq8kVlrGCcj4xumGd6w-C094XNnowcmZAKv-8-AHm1qHxAAgAA")

530

In [15]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/home/snrobot/lin/qwen2.5-7b")
tokens = tokenizer.encode(json.dumps(c, ensure_ascii=False, indent=4))
print(f"Token 数量: {len(tokens)}")

Token 数量: 4005


In [19]:
import json

# 完整的 JSON 字符串
c = '{ "response": { "status": true, "message": "Success", "timestamp": 1747651545933 }}'

# 解析 JSON 字符串
parsed_data = json.loads(c)
print(parsed_data)

{'response': {'status': True, 'message': 'Success', 'timestamp': 1747651545933}}


In [6]:
def trace_fields_from_response(api_chain):
    
    traces = []

    # 收集每一步之前所有响应的 observation.data
    previous_api_list = []
    for i, api_call in enumerate(api_chain):
        function_name = api_call.get('function_name')
        arguments = api_call.get('arguments', {})
        
        print(f"Processing API call {i + 1}: {function_name} with arguments: {arguments}")
        arg_sources = []

        for key, value in arguments.items():
            source_info = {"target_params_value": value}
            found = False

            for resp_index, previous_api in enumerate(previous_api_list):
                source_name = previous_api.get('function_name')
                prev_resp = previous_api.get('observation', {}).get('data', {})
                json_path, source_jsonpath_value = find_json_path(prev_resp, value) 
                if json_path:
                    source_info["target_api_params"] = key  
                    source_info["source_api"] = source_name
                    source_info["source_jsonpath"] = json_path
                    source_info["source_jsonpath_value"] = source_jsonpath_value
                    found = True
                    break

            if not found:
                source_info["target_api_params"] = key
                source_info["source_api"] = "query"
                source_info["source_jsonpath"] = None
                source_info["source_jsonpath_value"] = None
            
            arg_sources.append(source_info)
        
        traces.append({
            "function_name": function_name,
            "argument_sources": arg_sources
        })

        # 保存当前响应（用于后续调用参考）
        previous_api_list.append(api_call)
    return traces

def find_json_path(json_data, target_value):
    paths = []
    paths_value = []

    def search(obj, path="$"):
        if isinstance(obj, dict):
            for k, v in obj.items():
                search(v, f"{path}.{k}")
        elif isinstance(obj, list):
            for i, v in enumerate(obj):
                search(v, f"{path}[{i}]")
        else:
            if str(obj) == str(target_value):
                paths.append(path)
                paths_value.append(obj) 
            elif str(target_value) in str(obj):
                paths.append(path)
                paths_value.append(obj) 

    search(json_data)
    if paths:
        return paths[0], paths_value[0]
    else:
        return None, None

    

In [7]:
def trace_fields_from_response1(api_chain):
    traces = []
    previous_api_list = []

    for i, api_call in enumerate(api_chain):
        function_name = api_call.get('function_name')
        arguments = api_call.get('arguments', {})

        print(f"Processing API call {i + 1}: {function_name} with arguments: {arguments}")
        arg_sources = []

        def process_argument(key_path, value):
            source_info = {"target_params_value": value}
            found = False

            for previous_api in previous_api_list:
                source_name = previous_api.get('function_name')
                prev_args = previous_api.get('arguments', {})
                prev_resp = previous_api.get('observation', {}).get('data', {})

                # 如果 value 原本就是前面 API 的输入参数值 → 跳过
                if value_in_arguments(prev_args, value):
                    continue  # 不算来源


                json_path, source_jsonpath_value = find_json_path(prev_resp, value)
                if json_path:
                    source_info["target_api_params"] = key_path
                    source_info["source_api"] = source_name
                    source_info["source_jsonpath"] = json_path
                    source_info["source_jsonpath_value"] = source_jsonpath_value
                    found = True
                    break

            if not found:
                source_info["target_api_params"] = key_path
                source_info["source_api"] = "query"
                source_info["source_jsonpath"] = None
                source_info["source_jsonpath_value"] = None

            arg_sources.append(source_info)

        def traverse_arguments(obj, path=""):
            if isinstance(obj, dict):
                for k, v in obj.items():
                    new_path = f"{path}.{k}" if path else k
                    traverse_arguments(v, new_path)
            elif isinstance(obj, list):
                if obj:
                    # 只分析第一个元素，假设结构一致
                    new_path = f"{path}[0]"
                    traverse_arguments(obj[0], new_path)
            else:
                process_argument(path, obj)
        
        def value_in_arguments(arguments, target_value):
            found = False

            def search_args(obj):
                nonlocal found
                if found:
                    return
                if isinstance(obj, dict):
                    for v in obj.values():
                        search_args(v)
                elif isinstance(obj, list):
                    for item in obj:
                        search_args(item)
                else:
                    if str(obj) == str(target_value):
                        found = True

            search_args(arguments)
            return found



        traverse_arguments(arguments)

        traces.append({
            "function_name": function_name,
            "argument_sources": arg_sources
        })

        previous_api_list.append(api_call)

    return traces


In [8]:
def transform_get_room_availability(original_response):
    if original_response  in ({},[],"") or  "error" in original_response\
            or "bgo_error"  in original_response:
        return original_response
    # 提取原始数据
    lengths_of_stay = original_response.get("lengthsOfStay", [])
    currency = original_response.get("currency", "")
    av_dates = original_response.get("avDates", [])

    # 转换 lengthsOfStay
    transformed_lengths_of_stay = [
        {"date": list(item.keys())[0], "min_nights": list(item.values())[0]}
        for item in lengths_of_stay
    ]

    # 转换 avDates
    transformed_av_dates = [
        {"date": list(item.keys())[0], "price": list(item.values())[0]}
        for item in av_dates
    ]

    # 返回目标格式
    return {
        "currency": currency,
        "avDates": transformed_av_dates,
        "lengthsOfStay": transformed_lengths_of_stay
    }


In [9]:
#从静态响应中提取depends_on
import json 
file = open("/home/snrobot/lin/GraphLinkTools/Tools/Static_Response(1).json", "r")

data = json.load(file)

response = {}
for i, d in enumerate(data):
    query = d["related_query"]
    func_name = d["function_name"]
    if func_name == "Get_Room_Availability":
        d["observation"]["data"] = transform_get_room_availability(d["observation"]["data"]) 
   
    if query not in response:
        response[query] = []
    response[query].append(d)

api_chain_depends_on = {}
for idx, (query, res_list) in enumerate(response.items()):
    # if(idx > 0): break
    depens_on = trace_fields_from_response1(res_list)
    if depens_on:
        api_chain_depends_on[query] = depens_on

# 将结果写入文件
with open("/home/snrobot/lin/GraphLinkTools/gound_truth/api_link_gt.jsonl", "w", encoding="utf-8") as file_out:
    json.dump(api_chain_depends_on, file_out, ensure_ascii=False, indent=4)
#进一步合并
#=》》》》》》



Processing API call 1: Search_Car_Location with arguments: {'query': 'San Diego Marriott La Jolla'}
Processing API call 2: Search_Car_Rentals with arguments: {'pick_up_latitude': 32.873055, 'pick_up_longitude': -117.215935, 'drop_off_latitude': 32.873055, 'drop_off_longitude': -117.215935, 'pick_up_date': '2024-10-14', 'drop_off_date': '2024-10-15', 'pick_up_time': '08:00', 'drop_off_time': '08:00'}
Processing API call 3: Search_Car_Rentals with arguments: {'pick_up_latitude': 32.873055, 'pick_up_longitude': -117.215935, 'drop_off_latitude': 32.873055, 'drop_off_longitude': -117.215935, 'pick_up_date': '2024-10-15', 'drop_off_date': '2024-10-16', 'pick_up_time': '08:00', 'drop_off_time': '08:00'}
Processing API call 1: Search_Car_Location with arguments: {'query': 'Los Angeles International Airport'}
Processing API call 2: Search_Car_Rentals with arguments: {'pick_up_latitude': 33.9440994262695, 'pick_up_longitude': -118.402000427246, 'drop_off_latitude': 33.9440994262695, 'drop_off_lo

In [10]:
#对字段进行聚合操作
from collections import defaultdict
import json
file = open("/home/snrobot/lin/GraphLinkTools/gound_truth/api_link_gt.jsonl",'r')
data = json.load(file)
depens_on = defaultdict(lambda: defaultdict(list))

for query, gt_list in data.items():
    for gt in gt_list:
        func_name = gt["function_name"]
        argument_sources = gt["argument_sources"]

        for arg in argument_sources:
            target_param = arg["target_api_params"]
            source_api = arg["source_api"]
            if source_api == func_name:
                continue
            # 去重：检查是否已有相同 source_api 记录
            if all(existing["source_api"] != source_api for existing in depens_on[func_name][target_param]):
                depens_on[func_name][target_param].append(arg)
with open("/home/snrobot/lin/GraphLinkTools/gound_truth/api_link_gt(5).jsonl", "w", encoding="utf-8") as file_out:
    json.dump(depens_on, file_out, ensure_ascii=False, indent=4)
